This notebook reports on the test results for test `grain_size_strain_pinned.prm` and allows to compare the test against analytical results. The test models a small volume in simple shear flow with constant strain rate, and a grain size that is not at it's equilibrium value. We can compute how much the grain size should change according to the terms given in Eq. 8 of Mulyukova and Bercovici (2018):

\begin{equation}
\text{grain_growth} = \frac {3\eta_gk_g\sqrt h_g^{-q}}{(q R^{q-1})} \exp(\frac{-E_g}{R_gT}) \\
\text{grain_size_reduction} = - \frac{f h_g \Psi}{(3\eta_g)} R^2
\text{grain_size_change_rate} = \text{grain_growth} + \text{grain_size_reduction}
\text{grain_size_change} = \text{grain_size_change_rate} * \text{time_step_size}
\end{equation}

Here, $\eta_g$ is phase distribution function and $\sqrt{h_g} = 2/\pi$. The shear heating term $\Psi$ is $2\dot\varepsilon\tau$, where $\tau = 2 \eta \dot\varepsilon$, therefore, $\Psi = 4 \eta \dot\varepsilon ^2$. All other parameters are as given in Mulyukova and Bercovici (2018).

We can now for any given shear heating rate $\Psi$ compute the grain size change using the equations above. We can perform the same computation using the ASPECT model in the test `grain_size_strain_pinned.prm`. Then, we can compare the ASPECT model results to the analytical prediction, to confirm that we have correctly implemented the equations.


In [13]:
import numpy as np

These are function implementations of the terms above:

In [14]:

def growth_term(R):
  return (hg**(-q/2) * 3 * phase_distribution * grain_growth_prefactor * \
                                 np.exp(-grain_growth_activation/R_times_T))/(q*R**(q-1))
                      
def reduction_term(R, shear_heating):
  return f1*hg*shear_heating*(R**2)/(3*phase_distribution)

def shear_heating_rate(strain_rate, viscosity):
  return 4*strain_rate*strain_rate*viscosity

In [15]:
# define material constants here, making sure
# they align with the values in the test .prm file
q                       = 3.
grain_growth_prefactor  = 1.92e-10
grain_growth_activation = 4e5
f1                      = 1e-1
phase_distribution      = 0.6*0.4

hg         = (2/np.pi)*(2/np.pi)
R_times_T  = 8.314*1600

second_in_year = 3600*24*365.25
strain_rate    = 1e-5/second_in_year/2

# we run the test model for 1 year. The smaller the model length
# the more accurate the grain size evolution will be.
# We cannot run an instantaneous model though, because the grain size
# change rate has to be multiplied by the time step to compute a
# comparable output value.
dt = 1 * (3600*24*365.25)

In [16]:
# these are values predicted by ASPECT 
aspect_grain_size = 9.99897e-4
aspect_viscosity = 5.76833e17
aspect_grain_size_change = -1.02667e-7

In [17]:
# compute the expected values for grain_size_change_rate
# note that we use the ASPECT viscosity to compute the expected
# shear heating rate
shear_heating = shear_heating_rate(strain_rate, aspect_viscosity)
grain_size_change_rate = growth_term(aspect_grain_size) - reduction_term(aspect_grain_size, shear_heating)

# ASPECT reports change per timestep, not change rate, so multiply 
# the rate with the length of the timestep to create a comparable quantity.
expected_grain_size_change = grain_size_change_rate * dt

# Compare the expected with the actual change. Note that there is a difference between
# the two due to numerical accuracy. We tested experimentally that this difference decreases
# with decreasing time step size. 
print ("Expected theoretical change: ", expected_grain_size_change)
print ("Actual ASPECT change: ", aspect_grain_size_change)
print ("Relative difference: ", np.abs(aspect_grain_size_change - expected_grain_size_change) / expected_grain_size_change)


Expected theoretical change:  -1.0237701002326123e-07
Actual ASPECT change:  -1.02667e-07
Relative difference:  -0.002832569310950577
